# 한글-Claude-v2 Model: Abstractive Text Summarization with Amazon Titan
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.


## 개요
큰 문서로 작업할 때 입력 텍스트가 모델 컨텍스트 길이에 맞지 않거나 모델이 큰 문서로 착각하거나 메모리 부족 오류 등으로 인해 몇 가지 문제에 직면할 수 있습니다.

이러한 문제를 해결하기 위해 청킹 및 체인 프롬프트 개념을 기반으로 하는 아키텍처를 보여드리겠습니다. 이 아키텍처는 언어 모델로 구동되는 애플리케이션을 개발하는 데 널리 사용되는 프레임워크인 [LangChain](https://python.langchain.com/docs/get_started/introduction.html)을 활용합니다.

### 아키텍처

![](./images/42-text-summarization-2.png)

이 아키텍처에서:

1. 큰 문서(또는 작은 문서를 추가한 거대한 파일)가 로드됨
1. Langchain 유틸리티를 사용하여 여러 개의 작은 청크로 분할(청킹)
1. 첫 번째 청크가 모델로 전송됩니다. 모델은 해당 요약을 반환합니다.
1. Langchain은 다음 청크를 가져와 반환된 요약에 추가하고 결합된 텍스트를 새 요청으로 모델에 보냅니다. 모든 청크가 처리될 때까지 프로세스가 반복됩니다.
1. 결국 전체 내용을 기반으로 한 최종 요약이 있습니다.

### 사용 사례
이 접근 방식은 통화 기록, 회의 기록, 서적, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

## 1. Bedrock Client 생성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## 2. 긴 글 요약

### Boto3로 LangChain 구성

LangChain은 boto3 세션 정보를 LangChain에 전달하면 Bedrock에 액세스할 수 있습니다. None을 boto3 세션 정보로 LangChain에 전달하면 LangChain은 사용자 환경에서 세션 정보를 가져오려고 시도합니다.
올바른 클라이언트가 사용되도록 하기 위해 유틸리티 메서드 덕분에 클라이언트를 인스턴스화할 것입니다.

LangChain Bedrock 클래스에 대해 LLM을 지정해야 하며 추론을 위한 인수를 전달할 수 있습니다. 여기서 `model_id`에 Amazon Titan Text Large를 지정하고 `textGenerationConfig`에 Titan의 추론 파라미터를 전달합니다.

In [3]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# - create the Anthropic Model
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 4096,
        "temperature": 0.5,
        "top_k": 250,
        "top_p": 1,
        "stop_sequences": ["\n\nHuman"],        
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f7ad3eab400>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 4096, 'temperature': 0.5, 'top_k': 250, 'top_p': 1, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f7b11f9e140>])

## 3. 토큰이 많은 텍스트 파일 로드

`letters` 디렉토리에서 [2022년 주주들에게 보내는 아마존 CEO 편지](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter) 텍스트 파일을 찾을 수 있습니다. -주주에게). 다음 셀은 텍스트 파일을 로드하고 파일의 토큰 수를 계산합니다.

텍스트 파일의 토큰 수가 해당 모델의 최대 토큰 수를 초과했음을 나타내는 경고가 표시됩니다.

In [4]:
shareholder_letter = "./letters/2022-letter-kr.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

15829

## 4. 긴 텍스트를 덩어리로 나누기

텍스트가 너무 길어 프롬프트에 맞지 않으므로 더 작은 덩어리로 나눕니다.
LangChain의 `RecursiveCharacterTextSplitter`는 각 청크의 크기가 `chunk_size`보다 작아질 때까지 긴 텍스트를 재귀적으로 청크로 분할하는 것을 지원합니다. 텍스트는 `separators=["\n\n", "\n"]`로 청크로 구분되어 각 단락이 여러 청크로 분할되는 것을 방지합니다.

청크당 6,000자를 사용하여 각 부분에 대한 요약을 개별적으로 얻을 수 있습니다. 청크의 토큰 또는 단어 조각의 수는 텍스트에 따라 다릅니다.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [6]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

Now we have 5 documents and the first one has 3595 tokens


In [7]:
docs[0]

Document(page_content='CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 \u200b\u200b방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다. 그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.\n\n작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.\n\n제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다. 내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다. 오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다. 마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬가지였습니다. 거의 모든 책을 60초 안에 손끝에서 볼 수 있습니다 2007년 Kindle을 출시했을 때 가벼운 디지털 리더에 저장하고 검색할 수 있다는 것은 아직 "사물"이 아니었고 Alexa(2014년 출시)와 같이

## 5.청크 요약 및 결합

토큰의 수가 다른 문서에서 일관적이라고 가정하면 진행하는 것이 좋을 것입니다. LangChain의 [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html)을 사용하여 텍스트를 요약해 보겠습니다. `load_summarize_chain`은 `stuff`, `map_reduce` 및 `refine`의 세 가지 요약 방법을 제공합니다.
- `stuff`는 모든 청크를 하나의 프롬프트에 넣습니다. 따라서 이것은 토큰의 최대 한도에 도달하게 됩니다.
- `map_reduce`는 각 청크를 요약하고, 요약을 결합하고, 결합된 요약을 요약합니다. 결합된 요약이 너무 크면 오류가 발생합니다.
- 'refine'은 첫 번째 청크를 요약한 다음 첫 번째 요약으로 두 번째 청크를 요약합니다. 모든 청크가 요약될 때까지 동일한 프로세스가 반복됩니다.

`map_reduce` 및 `refine`은 LLM을 여러 번 호출하며 최종 요약을 얻는 데 시간이 걸립니다.
여기서 `map_reduce`를 사용해 봅시다.

In [8]:
from langchain import PromptTemplate

chuck_prompt_template = """
다음 텍스트의 요약을 한국어로 제공하십시오.
한 문장으로 대답해 주십시오.
TEXT: {text}
SUMMARY:
"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """
다음 텍스트를 한국어로 간결하게 요약하십시오.
텍스트의 요점을 다루는 글머리 기호로 응답을 반환합니다.
TEXT: {text}
SUMMARY:
"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

stuff_prompt_template = """
다음 텍스트의 요약을 제공하십시오.
TEXT: {text}
SUMMARY:
"""


In [9]:
def summary_chain_init(chain_type, llm):
    
    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            # return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            # return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

## 6. 요약 실행

In [10]:
# Set verbose=True if you want to see the prompts being used

from langchain.chains.summarize import load_summarize_chain
# summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)


# summary_chain = summary_chain_init(chain_type="REFINE", llm=llm)
summary_chain = summary_chain_init(chain_type="MAP_REDUCE", llm=llm)

> ⏰ **Note:** Depending on your number of documents, Bedrock request rate quota, and configured retry settings - this chain may take some time to run:

In [11]:
%%time 

output = summary_chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

다음 텍스트의 요약을 한국어로 제공하십시오.
한 문장으로 대답해 주십시오.
TEXT: CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 ​​방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다. 그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.

작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.

제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다. 내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다. 오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다. 마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬

In [12]:
print_ww(output.strip())

Here is a concise Korean summary in bullet points:

- 아마존은 팬데믹 기간 동안 온라인 쇼핑 증가에 힘입어 매출 성장
- 물류 인프라 확장, 네트워크 지역화 등 변화에 대응
- AWS는 단기적 둔화 예상하나 장기적으로 고객 중심 접근과 혁신을 통해 성장 가능
- 신규 사업 투자를 통해 고객 경험 개선 및 장기 성장 기회 모색
- 해외 시장 진출, 소매 사업 확장 등 기회 포착
- 인공지능, 클라우드컴퓨팅 등 신기술에 투자하여 혁신 추구
